### Rizky Ramdhani Koswara
### 11122300
### 4KA25

##**Importing Required Packages**


In [ ]:
!pip install mido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.0 MB/s eta 0:00:00


In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

##**Load MID file/files**

In [ ]:
!unzip /content/chillhopdata.zip -d chillhop

Archive:  /content/chillhopdata.zip
  inflating: chillhop/1.mid          
  inflating: chillhop/10.mid         
  inflating: chillhop/11.mid         
  inflating: chillhop/12.mid         
  inflating: chillhop/13.mid         
  inflating: chillhop/14.mid         
  inflating: chillhop/15.mid         
  inflating: chillhop/16.mid         
  inflating: chillhop/17.mid         
  inflating: chillhop/18.mid         
  inflating: chillhop/19.mid         
  inflating: chillhop/2.mid          
  inflating: chillhop/20.mid         
  inflating: chillhop/3.mid          
  inflating: chillhop/4.mid          
  inflating: chillhop/5.mid          
  inflating: chillhop/6.mid          
  inflating: chillhop/7.mid          
  inflating: chillhop/8.mid          
  inflating: chillhop/9.mid          
  inflating: chillhop/Cymatics - Eternity MIDI 1 - C Maj.mid  
  inflating: chillhop/Cymatics - Eternity MIDI 10 - F Min.mid  
  inflating: chillhop/Cymatics - Eternity MIDI 11 - A Maj.mid  
  inflating: 

Kode ini membaca file MIDI dari direktori tertentu, mengekstrak pitch dari setiap notasi yang dimainkan pada saluran pertama dengan tipe pesan `note_on`, dan menyimpannya dalam daftar `notes`.

In [ ]:
import os

notes = []
for song in os.listdir("/content/chillhop"):
  mid = MidiFile('/content/chillhop/' + song)
  for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type =='note_on':
      data = msg.bytes()
      notes.append(data[1])


##**Scale Data**

Kode ini menggunakan `MinMaxScaler` dari pustaka `sklearn` untuk menormalkan nilai pitch dari daftar `notes` agar berada dalam rentang [0, 1], dengan terlebih dahulu mengubah daftar `notes` menjadi array NumPy dan mereshape-nya menjadi vektor kolom (-1, 1) sebelum proses transformasi.

In [ ]:
scaler = MinMaxScaler()
notes = list(scaler.fit_transform(np.array(notes).reshape(-1, 1)))

##**Create Train Data**

kode ini membagi data notasi musik (notes) menjadi urutan input-output, di mana input terdiri dari 30 notasi sebelumnya untuk memprediksi notasi berikutnya, dan memisahkan sebagian data untuk digunakan sebagai data uji.

In [ ]:
notes = [list(note) for note in notes]

X = []
y = []

n_prev=30

for i in range(len(notes) - n_prev):
  X.append(notes[i:i+n_prev])
  y.append(notes[i+n_prev])

X_test = X[-300:]
X = X[:-300]
y = y[:-300]

##**Build LSTM**

model ini menggunakan tiga lapisan LSTM untuk memproses urutan notasi musik, dengan dropout di setiap lapisan untuk mencegah overfitting, dan menghasilkan prediksi nilai kontinu (pitch) menggunakan lapisan Dense dengan aktivasi linear.

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
model.summary()

optimizer = Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=optimizer)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 30, 256)             │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 30, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 30, 128)             │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 30, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 510,785 (1.95 MB)

 Trainable params: 510,785 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

##**Training**

Melatih model menggunakan data X dan y yang telah dipersiapkan, dengan ukuran batch 16 dan 10 epoch, serta menampilkan progres pelatihan selama proses berlangsung.

In [ ]:
model.fit(np.array(X), np.array(y), batch_size=16, epochs=10, verbose=1)

Epoch 1/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 26s 135ms/step - loss: 0.0178
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 41s 135ms/step - loss: 0.0165
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 41s 137ms/step - loss: 0.0157
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 41s 136ms/step - loss: 0.0150
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 41s 136ms/step - loss: 0.0153
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 27s 137ms/step - loss: 0.0161
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 43s 145ms/step - loss: 0.0149
Epoch 8/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 39s 136ms/step - loss: 0.0146
Epoch 9/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 27s 136ms/step - loss: 0.0154
Epoch 10/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 40s 133ms/step - loss: 0.0147


##**Generating & Saving LSTM Music**

Kode ini mengubah hasil prediksi model menjadi notasi MIDI yang dapat dimainkan, dan menyimpannya dalam file LSTM_music.mid.

In [ ]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music.mid')

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step
